MIT License

Copyright (c) 2021 Taiki Miyagawa and Akinori F. Ebihara

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

### For Optuna.version>0.14, some modifications needed.

In [1]:
import os
import numpy as np
import sqlite3
from itertools import zip_longest

In [2]:
########## User defined parameters #########
dblist = [
    "./data-directory/example_NMNIST-H.db",
    "./data-directory/example_UCF101.db",
]

In [3]:
########## User defined parameters #########
name_db = dblist[0]

keys = ["learning_rate", "optimizer", "weight_decay", "dropout", "batch_size", "param_llr_loss", "order_sprt"] 
    # For train_ti_nmnist-h.py.
    # Use this for "./data-directory/example_NMNIST-H.db".
#keys = ["learning_rate", "optimizer", "weight_decay", "dropout", "batch_size", "param_llr_loss", "order_sprt", "beta"] 
    # For train_ti_UCF101.py.
    # Use this for "./data-directory/example_UCF101.db".
#keys = ["learning_rate", "optimizer", "weight_decay", "batch_size", "param_multLam", "order_sprt"] 
    # For train_dre_nmnist-h.py.
#keys = ["learning_rate", "optimizer", "weight_decay", "batch_size", "param_multLam", "order_sprt", "beta"] 
    # For train_dre_UCF101.py.
############################################ end #
nb_params = len(keys)
maxnum_show = 1000

# Get connect cursor
assert os.path.exists(name_db), name_db
con = sqlite3.connect(name_db)
c = con.cursor()

# Get count
count = c.execute("select count(value) from trials").fetchall()[0][0]
raw = c.execute("select * from trials").fetchall()
_count = count

# Get and cleanse trial params 
trials = c.execute("select * from trials order by value asc").fetchall()
trial_params = c.execute("select * from trial_params").fetchall() 

ls_idx = []
for trial in trials:
    if trial[2] == "FAIL":
        continue

    else:
        ls_idx.append([trial[0], trial[3]]) # [int, float] = [trial num, best value]
        if len(ls_idx) == maxnum_show:
            break
            
ls_show = []            
for iter_idx in ls_idx:
    for iter_params in trial_params:
        # Extract tirial params that showed good results
        if iter_params[1] == iter_idx[0]:
            ls_show.append([iter_idx[0], iter_idx[1], iter_params[2], iter_params[3]])

# Show search spaces, and some sanity checks of 'keys'
print("Search Spaces")
assert_set = set()
for i in range(nb_params):
    print(trial_params[i][2:])
    key = trial_params[i][2]
    assert key in keys, "Maybe there is another keyword that should be included in 'keys'. See 'trial_params'."
    if assert_set >= {key}:
        raise ValueError("Maybe there is an irrelevant keyword in 'keys' (or a typo?).")        
    assert_set.add(trial_params[i][2])
    
if not (assert_set >= {trial_params[i+1][2]}):
    raise ValueError("Maybe there are more keywords that should be included in 'keys'. See 'trial_params'.")
    
# Show top parameters
dc = dict()
for key in keys:
    dc[key] = []

for itr_show in ls_show:
    for key in keys:
        if itr_show[-2] == key:
            dc[key].append(itr_show)     
            
    
# Show trial params
###################
print("\nNum trials: {}".format(_count))
print("trial number, best value\n               {}".format(keys))

for i in range(len(dc[keys[0]])):
    for j, key in enumerate(keys):
        result = dc[key]

        if result[i][1] is None:
            continue
        
        if j == 0:
            print("{:4d} {:.7f}".format(result[i][0], result[i][1]), "          ", end="")

        print("{}          ".format(result[i][3]), end="")

    if result[i][1] is not None:            
        print("")
        

Search Spaces
('learning_rate', 0.0, '{"attributes": {"choices": [0.01, 0.001, 0.0001]}, "name": "CategoricalDistribution"}')
('batch_size', 0.0, '{"attributes": {"choices": [500]}, "name": "CategoricalDistribution"}')
('optimizer', 1.0, '{"attributes": {"choices": ["adam", "rmsprop"]}, "name": "CategoricalDistribution"}')
('dropout', 0.0, '{"attributes": {"choices": [0.0]}, "name": "CategoricalDistribution"}')
('weight_decay', 1.0, '{"attributes": {"choices": [0.001, 0.0001, 1e-05]}, "name": "CategoricalDistribution"}')
('param_llr_loss', 4.0, '{"attributes": {"choices": [0.1, 1.0, 10.0, 100.0, 1000]}, "name": "CategoricalDistribution"}')
('order_sprt', 4.0, '{"attributes": {"choices": [0, 1, 5, 10, 15, 19]}, "name": "CategoricalDistribution"}')

Num trials: 500
trial number, best value
               ['learning_rate', 'optimizer', 'weight_decay', 'dropout', 'batch_size', 'param_llr_loss', 'order_sprt']
 359 0.1430468           2.0          1.0          1.0          0.0          0.0  

 256 0.1437234           2.0          1.0          1.0          0.0          0.0          0.0          2.0          
 107 0.1437236           2.0          1.0          1.0          0.0          0.0          1.0          2.0          
   9 0.1437306           2.0          1.0          2.0          0.0          0.0          3.0          4.0          
 392 0.1437353           2.0          1.0          1.0          0.0          0.0          3.0          5.0          
 149 0.1437353           2.0          1.0          2.0          0.0          0.0          1.0          2.0          
 397 0.1437376           2.0          1.0          1.0          0.0          0.0          3.0          5.0          
 170 0.1437379           2.0          1.0          1.0          0.0          0.0          1.0          2.0          
 116 0.1437402           2.0          1.0          1.0          0.0          0.0          2.0          2.0          
 161 0.1437407           2.0          1.0          1.0          

  58 0.1682552           2.0          1.0          1.0          0.0          0.0          4.0          0.0          
  57 0.1687370           2.0          1.0          1.0          0.0          0.0          4.0          0.0          
  60 0.1688266           2.0          1.0          1.0          0.0          0.0          4.0          0.0          
  10 0.1793680           0.0          1.0          2.0          0.0          0.0          2.0          0.0          


In [4]:
# select * from trials
print("raw trial data")
raw

raw trial data


[(1,
  1,
  'COMPLETE',
  0.15071768222579607,
  '2020-12-08 10:52:37.849557',
  '2020-12-08 12:08:52.886813'),
 (2,
  1,
  'COMPLETE',
  0.16302771995214127,
  '2020-12-08 10:52:40.574155',
  '2020-12-08 11:33:09.826526'),
 (3,
  1,
  'COMPLETE',
  0.1439744365360458,
  '2020-12-08 10:52:43.707091',
  '2020-12-08 12:10:11.734945'),
 (4,
  1,
  'COMPLETE',
  0.15173965914636867,
  '2020-12-08 10:52:46.894294',
  '2020-12-08 12:10:48.450549'),
 (5,
  1,
  'COMPLETE',
  0.14400323640918522,
  '2020-12-08 10:52:49.973438',
  '2020-12-08 12:10:20.523109'),
 (6,
  1,
  'COMPLETE',
  0.15008114756157886,
  '2020-12-08 10:52:53.126063',
  '2020-12-08 12:19:41.169836'),
 (7,
  1,
  'COMPLETE',
  0.14403575363438836,
  '2020-12-08 10:52:55.785622',
  '2020-12-08 12:00:38.199900'),
 (8,
  1,
  'COMPLETE',
  0.14574162302104343,
  '2020-12-08 10:55:09.150289',
  '2020-12-08 11:57:38.802550'),
 (9,
  1,
  'COMPLETE',
  0.14373056405841922,
  '2020-12-08 10:55:12.561941',
  '2020-12-08 12:27:48.524

In [5]:
# select * from trials order by value asc
print("All trials")
trials

All trials


[(359,
  1,
  'COMPLETE',
  0.1430468256949583,
  '2020-12-10 00:21:24.437469',
  '2020-12-10 01:34:05.722491'),
 (276,
  1,
  'COMPLETE',
  0.14307135495623524,
  '2020-12-09 16:02:46.642237',
  '2020-12-09 16:57:39.075766'),
 (175,
  1,
  'COMPLETE',
  0.14312462098287215,
  '2020-12-09 06:45:51.988685',
  '2020-12-09 07:41:09.935115'),
 (44,
  1,
  'COMPLETE',
  0.14314467358229055,
  '2020-12-08 15:28:15.576004',
  '2020-12-08 16:26:11.754357'),
 (330,
  1,
  'COMPLETE',
  0.14318603221463777,
  '2020-12-09 21:12:30.919734',
  '2020-12-09 22:30:16.825327'),
 (482,
  1,
  'COMPLETE',
  0.14320027503998745,
  '2020-12-10 13:46:12.711554',
  '2020-12-10 14:53:50.605949'),
 (185,
  1,
  'COMPLETE',
  0.14320870436869682,
  '2020-12-09 07:38:24.789470',
  '2020-12-09 08:31:56.561286'),
 (154,
  1,
  'COMPLETE',
  0.14323403212155328,
  '2020-12-09 04:16:24.730619',
  '2020-12-09 05:02:38.251441'),
 (165,
  1,
  'COMPLETE',
  0.1432399673975887,
  '2020-12-09 05:14:36.558079',
  '2020-12

In [6]:
# select * from trial_params
print("All trial parameters")
trial_params[:]

All trial parameters


[(1,
  1,
  'learning_rate',
  0.0,
  '{"attributes": {"choices": [0.01, 0.001, 0.0001]}, "name": "CategoricalDistribution"}'),
 (2,
  1,
  'batch_size',
  0.0,
  '{"attributes": {"choices": [500]}, "name": "CategoricalDistribution"}'),
 (3,
  1,
  'optimizer',
  1.0,
  '{"attributes": {"choices": ["adam", "rmsprop"]}, "name": "CategoricalDistribution"}'),
 (4,
  1,
  'dropout',
  0.0,
  '{"attributes": {"choices": [0.0]}, "name": "CategoricalDistribution"}'),
 (5,
  1,
  'weight_decay',
  1.0,
  '{"attributes": {"choices": [0.001, 0.0001, 1e-05]}, "name": "CategoricalDistribution"}'),
 (6,
  1,
  'param_llr_loss',
  4.0,
  '{"attributes": {"choices": [0.1, 1.0, 10.0, 100.0, 1000]}, "name": "CategoricalDistribution"}'),
 (7,
  1,
  'order_sprt',
  4.0,
  '{"attributes": {"choices": [0, 1, 5, 10, 15, 19]}, "name": "CategoricalDistribution"}'),
 (8,
  2,
  'learning_rate',
  2.0,
  '{"attributes": {"choices": [0.01, 0.001, 0.0001]}, "name": "CategoricalDistribution"}'),
 (9,
  2,
  'batc